In [19]:
import pandas as pd
import oracledb
from tabulate import tabulate
import random
import re
import webbrowser
 
#SCRE
username = 'svcGOCUI'
password = 'DELL2023support#'
host = 'gocplorlvpr18.amer.dell.com'
port = '1521'
service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'
 
#DAO GE4 SIT
# dbname="DAO_GE4"
# username = 'SIT_BROWSER'
# password = 'browse_s1t'
# host = 'udmnlorrrsi2a01.amer.dell.com'
# port = '1521'
# service_name = 'fdr4s.sit.amer.dell.com'

def Retrive_VectorData(region):
    region =region.upper()
    excel_name="Input_Order_Incoming_Data.xlsx"
    dsn = oracledb.makedsn(host, port, service_name=service_name)
    try :
        print("conn")
        connection = oracledb.connect(user=username, password=password, dsn=dsn)   
        print("Connected to Oracle Database")
        cursor = connection.cursor()
        # query = '''SELECT t.owner || '.' || t.table_name AS Table_Name,c.column_name as Column_Name FROM all_tables t 
        # JOIN all_tab_columns c ON t.table_name = c.table_name  ORDER BY t.table_name, c.column_name'''
 
        query = f'''select a.region, to_char(a.processdate_utc,'yyyymmdd') processday,to_char(a.processdate_utc,'hh24') as processday_hour,
a.order_count,b.fiscalyear,b.fiscalquarter,b.fiscalmonth,b.fiscalweek,to_char(a.processdate_utc,'Day') as DayNum
  from work.tb_orderprocess_effectiveness a,work.tb_fiscalcalendar b
where to_char(a.processdate_utc,'yyyymmdd') between b.startdate and b.enddate
   and a.functionname = 'Order Incoming'
   and datatype = 'NEW'
   and vendor = 'ALL'
   AND REGION = '{region}'
order by a.processdate_utc asc'''
 
        cursor.execute(query)
 
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        data = pd.DataFrame(rows,columns=column_names)
        print(excel_name)
        data.to_excel(excel_name,index=False)
        table = tabulate(rows, headers=column_names, tablefmt="pretty")
        print(data)
        return data
    except Exception as e:
       print(f"An error occurred: {e}")
    finally:
        if connection:
            connection.close()
            print("Connection closed")

In [24]:
region = 'apj'
region = region.upper()
data = Retrive_VectorData(region)

conn
Connected to Oracle Database
Input_Order_Incoming_Data.xlsx
     REGION PROCESSDAY PROCESSDAY_HOUR  ORDER_COUNT FISCALYEAR FISCALQUARTER  \
0       APJ   20240207              09          508       FY25            Q1   
1       APJ   20240207              10          377       FY25            Q1   
2       APJ   20240207              11          264       FY25            Q1   
3       APJ   20240207              12          146       FY25            Q1   
4       APJ   20240207              13           97       FY25            Q1   
...     ...        ...             ...          ...        ...           ...   
3938    APJ   20240729              06          635       FY25            Q2   
3939    APJ   20240729              07          869       FY25            Q2   
3940    APJ   20240729              08          776       FY25            Q2   
3941    APJ   20240729              09          799       FY25            Q2   
3942    APJ   20240729              10          502    

In [25]:
data.head()

,REGION,PROCESSDAY,PROCESSDAY_HOUR,ORDER_COUNT,FISCALYEAR,FISCALQUARTER,FISCALMONTH,FISCALWEEK,DAYNUM
0,APJ,20240207,09,508,FY25,Q1,M01,WK01,Wednesday
1,APJ,20240207,10,377,FY25,Q1,M01,WK01,Wednesday
2,APJ,20240207,11,264,FY25,Q1,M01,WK01,Wednesday
3,APJ,20240207,12,146,FY25,Q1,M01,WK01,Wednesday
4,APJ,20240207,13,97,FY25,Q1,M01,WK01,Wednesday


In [ ]:
# # Uploading out put file into SQL DB
# import pandas as pd
# from sqlalchemy import create_engine
# from sqlalchemy.types import Integer, String, DateTime, Numeric
# import cx_Oracle
 
# #SCRE
# username = 'svcGOCUI'
# password = 'DELL2023support#'
# host = 'gocplorlvpr18.amer.dell.com'
# port = '1521'
# service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'
# dsn = oracledb.makedsn(host, port, service_name=service_name)
 
# engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{dsn}')
# dtype = {
#     'REGION': String(256), 
#     'FORECAST_DATE': DateTime,
#     'MODEL1': Numeric,
#     'MODEL2': Numeric,
#     'MODEL3': Numeric,
#     'MODEL4': Numeric,
#     'MODEL5': Numeric,
#     'ACTUALORDERUNIT': Numeric,
#     'CREATED_DATE': DateTime,
#     'UPDATED_DATE': DateTime,
# }
 
# try:
#     data = pd.read_csv(r"C:/Users/S_Munwar/Downloads/Final_Model_for_OIF/Future_forecasted_OIF_SQL.csv")
#     print(type(data))
#     data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'])
#     data['CREATED_DATE'] = pd.to_datetime(data['CREATED_DATE'])
#     data['UPDATED_DATE'] = pd.to_datetime(data['UPDATED_DATE'])
#     print(type(data))
#     data.to_sql('tb_orderprocess_effectiveness_forecast',schema ='work', con=engine, if_exists='append', index=False)
#     print("added successfully")
# except Exception as e:
#     print(f"An error occurred: {e}")
# finally:
#     engine.dispose()
#     print("Connection closed")

In [28]:
import pandas as pd

# Convert PROCESSDAY to proper date format
data["PROCESSDAY"] = pd.to_datetime(data["PROCESSDAY"], format="%Y%m%d")

data["PROCESSDAY_HOUR"] = data["PROCESSDAY_HOUR"].astype(int)
# Create a new column combining PROCESSDAY and PROCESSDAY_HOUR
data["DateTime"] = data["PROCESSDAY"] + pd.to_timedelta(data["PROCESSDAY_HOUR"], unit="h")

# Drop the index column
#data = data.reset_index(drop=True)

In [32]:
#df= data.drop(columns=['PROCESSDAY', 'PROCESSDAY'])
df=data
print(region)

APJ


In [44]:
# Separate dataframes for each region

df = df[df["REGION"] == region]
# emea_df = df[df["REGION"] == "EMEA"]
# dao_df = df[df["REGION"] == "DAO"]
# apj_df = df[df["REGION"] == "APJ"]

# Print the dataframes
print(f"{region}  DataFrame:")
print(df)
# print("\nDAO DataFrame:")
# print(dao_df)
# print("\nAPJ DataFrame:")
# print(apj_df)

APJ  DataFrame:
     REGION PROCESSDAY  PROCESSDAY_HOUR  ORDER_COUNT FISCALYEAR FISCALQUARTER  \
0       APJ 2024-02-07                9          508       FY25            Q1   
1       APJ 2024-02-07               10          377       FY25            Q1   
2       APJ 2024-02-07               11          264       FY25            Q1   
3       APJ 2024-02-07               12          146       FY25            Q1   
4       APJ 2024-02-07               13           97       FY25            Q1   
...     ...        ...              ...          ...        ...           ...   
3938    APJ 2024-07-29                6          635       FY25            Q2   
3939    APJ 2024-07-29                7          869       FY25            Q2   
3940    APJ 2024-07-29                8          776       FY25            Q2   
3941    APJ 2024-07-29                9          799       FY25            Q2   
3942    APJ 2024-07-29               10          502       FY25            Q2   

     FISCAL

In [56]:
# Missing Values imputations for 3 regions
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Assuming df is your original DataFrame with all regions
df = df[df["REGION"] == region]

# emea_df = df[df["REGION"] == "EMEA"]
# dao_df = df[df["REGION"] == "DAO"]
# apj_df = df[df["REGION"] == "APJ"]

# Define a function for imputation
def perform_imputation(df, region_name):
    # Convert DateTime column to datetime format
    df["DateTime"] = pd.to_datetime(df["DateTime"])

    # Generate 1-hour intervals
    start_date = df["DateTime"].min()
    end_date = df["DateTime"].max()
    hour_intervals = pd.date_range(start=start_date, end=end_date, freq="H")

    # Create a new DataFrame with intervals
    interval_df = pd.DataFrame({"DateTime": hour_intervals})

    # Merge interval_df with original data
    merged_df = pd.merge(interval_df, df, on="DateTime", how="left")

    # Fill missing data based on available data
    merged_df["REGION"].fillna(method="ffill", inplace=True)
    merged_df["FISCALYEAR"].fillna(method="ffill", inplace=True)
    merged_df["FISCALQUARTER"].fillna(method="ffill", inplace=True)
    merged_df["FISCALMONTH"].fillna(method="ffill", inplace=True)
    merged_df["FISCALWEEK"].fillna(method="ffill", inplace=True)
    merged_df["DAYNUM"].fillna(method="ffill", inplace=True)

    # Perform imputation for ORDER_COUNT using DAYNUM
    daynum_order_count_avg = merged_df.groupby("DAYNUM")["ORDER_COUNT"].mean()
    merged_df["ORDER_COUNT"].fillna(merged_df["DAYNUM"].map(daynum_order_count_avg), inplace=True)

    # Fill remaining columns based on data & time columns
    merged_df["PROCESSDAY_HOUR"].fillna(merged_df["DateTime"].dt.hour, inplace=True)

    # Save the imputed DataFrame to an Excel file
    # output_path = f"C:\\Users\\S_Munwar\\Downloads\\Final_Model_for_OIF\\{region_name}_imputated.xlsx"
    return merged_df
    

# Perform imputation for each region
# for region_df, region_name in zip([emea_df, dao_df, apj_df], ["EMEA", "DAO", "APJ"]):
#     print(region_name)
#     perform_imputation(region_df, region_name)

print(region)
imputed_df = perform_imputation(df, region)

print(imputed_df)
imputed_df.to_excel(f"{region}_imputated.xlsx", index=False)
# Print the resulting DataFrame
print(f"Imputed data for {region} saved")

APJ
                DateTime REGION PROCESSDAY  PROCESSDAY_HOUR  ORDER_COUNT  \
0    2024-02-07 09:00:00    APJ 2024-02-07              9.0        508.0   
1    2024-02-07 10:00:00    APJ 2024-02-07             10.0        377.0   
2    2024-02-07 11:00:00    APJ 2024-02-07             11.0        264.0   
3    2024-02-07 12:00:00    APJ 2024-02-07             12.0        146.0   
4    2024-02-07 13:00:00    APJ 2024-02-07             13.0         97.0   
...                  ...    ...        ...              ...          ...   
4149 2024-07-29 06:00:00    APJ 2024-07-29              6.0        635.0   
4150 2024-07-29 07:00:00    APJ 2024-07-29              7.0        869.0   
4151 2024-07-29 08:00:00    APJ 2024-07-29              8.0        776.0   
4152 2024-07-29 09:00:00    APJ 2024-07-29              9.0        799.0   
4153 2024-07-29 10:00:00    APJ 2024-07-29             10.0        502.0   

     FISCALYEAR FISCALQUARTER FISCALMONTH FISCALWEEK     DAYNUM  
0          FY25  

C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_33264\2834530333.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hour_intervals = pd.date_range(start=start_date, end=end_date, freq="H")
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_33264\2834530333.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df["REGION"].fillna(method="ffill", inplace=True)
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_33264\2834530333.py:30: FutureWarn

Imputed data for APJ saved


In [57]:
imputed_df.head()

,DateTime,REGION,PROCESSDAY,PROCESSDAY_HOUR,ORDER_COUNT,FISCALYEAR,FISCALQUARTER,FISCALMONTH,FISCALWEEK,DAYNUM
0,2024-02-07 09:00:00,APJ,2024-02-07,9.0,508.0,FY25,Q1,M01,WK01,Wednesday
1,2024-02-07 10:00:00,APJ,2024-02-07,10.0,377.0,FY25,Q1,M01,WK01,Wednesday
2,2024-02-07 11:00:00,APJ,2024-02-07,11.0,264.0,FY25,Q1,M01,WK01,Wednesday
3,2024-02-07 12:00:00,APJ,2024-02-07,12.0,146.0,FY25,Q1,M01,WK01,Wednesday
4,2024-02-07 13:00:00,APJ,2024-02-07,13.0,97.0,FY25,Q1,M01,WK01,Wednesday


In [59]:
import pandas as pd

# Define a function to select specific columns from a DataFrame
def select_columns(df, columns):
    return df[columns].reset_index(drop=True)

# Define the columns to select
selected_columns = ["DateTime", "ORDER_COUNT"]

# Assuming emea_df, dao_df, and apj_df are your DataFrames for the EMEA, DAO, and APJ regions respectively
# Apply the function to each DataFrame
data_df = select_columns(imputed_df, selected_columns)

# data_emea = select_columns(emea_df, selected_columns)
# data_dao = select_columns(dao_df, selected_columns)
# data_apj = select_columns(apj_df, selected_columns)

# Now you have the selected columns for each region
print(f"Selected columns for {region}:")
print(data_df.head())

# print("\nSelected columns for DAO:")
# print(data_dao.head())

# print("\nSelected columns for APJ:")
# print(data_apj.head())


Selected columns for APJ:
             DateTime  ORDER_COUNT
0 2024-02-07 09:00:00        508.0
1 2024-02-07 10:00:00        377.0
2 2024-02-07 11:00:00        264.0
3 2024-02-07 12:00:00        146.0
4 2024-02-07 13:00:00         97.0


In [60]:
# Forecasting Models for 3 Regions
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

def process_region(data, region_name):
    # Feature engineering for seasonality
    data['hour'] = data.index.hour
    data['dayofweek'] = data.index.dayofweek
    data['month'] = data.index.month

    # Prepare features and target with seasonality
    X = data[['hour', 'dayofweek', 'month']]  # Add more features as needed
    y = data['ORDER_COUNT']

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train your models here (Decision Tree, Random Forest, Gradient Boosting, SVM)
    # Decision Tree Regressor
    dt_model = DecisionTreeRegressor(random_state=42)
    dt_model.fit(X_train, y_train)
    dt_predictions = dt_model.predict(X_test)
    dt_mse = mean_squared_error(y_test, dt_predictions)
    print(f'{region_name} - Decision Tree MSE: {dt_mse}')

    # Random Forest Regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_predictions = rf_model.predict(X_test)
    rf_mse = mean_squared_error(y_test, rf_predictions)
    print(f'{region_name} - Random Forest MSE: {rf_mse}')

    # Gradient Boosting Regressor
    gbm_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    gbm_model.fit(X_train, y_train)
    gbm_predictions = gbm_model.predict(X_test)
    gbm_mse = mean_squared_error(y_test, gbm_predictions)
    print(f'{region_name} - GBM MSE: {gbm_mse}')

    # Support Vector Machine Regressor
    svm_model = SVR(kernel='rbf')
    svm_model.fit(X_train, y_train)
    svm_predictions = svm_model.predict(X_test)
    svm_mse = mean_squared_error(y_test, svm_predictions)
    print(f'{region_name} - SVM MSE: {svm_mse}')

    # Combine actual and forecasted values into DataFrames for comparison
    actual_vs_forecasted_dt = pd.DataFrame({'Actual': y_test, 'Forecasted_DT': dt_predictions})
    actual_vs_forecasted_rf = pd.DataFrame({'Actual': y_test, 'Forecasted_RF': rf_predictions})
    actual_vs_forecasted_gbm = pd.DataFrame({'Actual': y_test, 'Forecasted_GBM': gbm_predictions})
    actual_vs_forecasted_svm = pd.DataFrame({'Actual': y_test, 'Forecasted_SVM': svm_predictions})

    # Extend the datetime index for the next 100 future values with 1-hour interval
    last_timestamp = data.index[-1]
    future_dates = pd.date_range(start=last_timestamp, periods=167, freq='H')[1:]

    # Create a DataFrame for future dates and engineer seasonality features
    future_df = pd.DataFrame(index=future_dates)
    future_df['hour'] = future_df.index.hour
    future_df['dayofweek'] = future_df.index.dayofweek
    future_df['month'] = future_df.index.month

    # Predict the next 100 future values using each model
    future_predictions_dt = dt_model.predict(future_df)
    future_predictions_rf = rf_model.predict(future_df)
    future_predictions_gbm = gbm_model.predict(future_df)
    future_predictions_svm = svm_model.predict(future_df)

    # Create a DataFrame for the future predictions
    future_predictions_df = pd.DataFrame({
        'DateTime': future_dates,
        'Forecasted_DT': future_predictions_dt,
        'Forecasted_RF': future_predictions_rf,
        'Forecasted_GBM': future_predictions_gbm,
        'Forecasted_SVM': future_predictions_svm
    })

    # Set the DateTime as the index
    future_predictions_df.set_index('DateTime', inplace=True)

    # Combine actual vs forecasted and future forecasted values into a single DataFrame
    combined_results_df = pd.concat([
        actual_vs_forecasted_dt,
        actual_vs_forecasted_rf[['Forecasted_RF']],
        actual_vs_forecasted_gbm[['Forecasted_GBM']],
        actual_vs_forecasted_svm[['Forecasted_SVM']],
        future_predictions_df
    ], axis=1)

    # Define the output path for the Excel file
    output_excel_path = f'ML_forecast_results_{region_name}_.xlsx'

    # Export the DataFrame to an Excel file
    combined_results_df.to_excel(output_excel_path, index=True)

    print(f"Data exported to {output_excel_path} for {region_name}")

# Load your datasets
data_df = pd.DataFrame(data_df)
data_df['DateTime'] = pd.to_datetime(data_df['DateTime'])
data_df.set_index('DateTime', inplace=True)




# Process each region
process_region(data_df, region)



APJ - Decision Tree MSE: 26438.653661910306
APJ - Random Forest MSE: 25471.165728655287
APJ - GBM MSE: 23781.485937879428
APJ - SVM MSE: 90515.51213771291


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_33264\2146460868.py:60: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=last_timestamp, periods=167, freq='H')[1:]


Data exported to ML_forecast_results_APJ_.xlsx for APJ
